
Right click and open the link in new tab to open the notebook in google colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KenR22/twitter_spark_nlp/blob/master/bds_project.ipynb)

# Download the Data

Upload your Kaggle.json file here if you want to download the data directly from Kaggle. To avaoid downloading data from Kaggle, upload the csv file under the data folder in github repo:

In [1]:
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!kaggle
!mv kaggle.json ../root/.kaggle/kaggle.json

!kaggle datasets download -d ayushggarg/all-trumps-twitter-insults-20152021
!unzip all-trumps-twitter-insults-20152021.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
  0% 0.00/581k [00:00<?, ?B/s]
100% 581k/581k [00:00<00:00, 39.4MB/s]
Archive:  all-trumps-twitter-insults-20152021.zip
  inflating: trump_insult_tweets_2014_to_2021.csv  


# Imports


In [3]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-05-05 16:04:23--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-05-05 16:04:23--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-05-05 16:04:23 (29.3 MB/s) - written to stdout [1608/1608]

setup Cola

In [4]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
spark = sparknlp.start()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import to_date,month, year


import plotly.graph_objs as go
import pandas as pd
import requests

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context

spark = SparkSession.builder.getOrCreate()

## Load the Data
If the download from Kaggle worked then the csv file should be in working directory. The csv is loaded as RDD below. There is one date column that was converted to date format from string manually.

In [6]:
df = spark.read.format("csv").load("trump_insult_tweets_2014_to_2021.csv",
                                         header='true', 
                                         inferSchema='false')
df = df.select(to_date(df.date, 'yyyy-MM-dd').alias('date'),"target","insult","tweet")

df.printSchema()


root
 |-- date: date (nullable = true)
 |-- target: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- tweet: string (nullable = true)



Show the first 5 rows of the RDD

In [7]:
df.show(5)

+----------+--------------+--------------------+--------------------+
|      date|        target|              insult|               tweet|
+----------+--------------+--------------------+--------------------+
|2014-10-09|thomas-frieden|                fool|"Can you believe ...|
|2014-10-09|thomas-frieden|                DOPE|"Can you believe ...|
|2015-06-16|   politicians|all talk and no a...|Big time in U.S. ...|
|2015-06-24|    ben-cardin|It's politicians ...|Politician @Senat...|
|2015-06-24|    neil-young|     total hypocrite|For the nonbeliev...|
+----------+--------------+--------------------+--------------------+
only showing top 5 rows



In [8]:
print("Total Number of Tweets: {0} ".format(df.count()))

Total Number of Tweets: 10492 


That is a lot of insult in 5 years!


# Targets

How many targets did he have in his tweets? Were they all the similar?

Let's group by the target column and get the counts.

In [9]:
count_target=df.groupBy("target").agg(count("*").alias("Number of Tweet")).sort(desc("Number of Tweet")).filter(col("Number of Tweet")>120)

In [10]:
count_target.show()


+-------------------+---------------+
|             target|Number of Tweet|
+-------------------+---------------+
|          the-media|           1287|
|          democrats|            647|
|    hillary-clinton|            625|
|       trump-russia|            441|
|          joe-biden|            402|
| the-new-york-times|            392|
|                cnn|            295|
|impeachment-inquiry|            247|
|        adam-schiff|            143|
|      2020-election|            143|
|       nancy-pelosi|            129|
|               null|            123|
|        james-comey|            122|
+-------------------+---------------+



In [11]:
import plotly.express as px

In [12]:
fig = px.bar(count_target.toPandas(), x='target', y='Number of Tweet')
fig.show()

The biggest target of Trump's tweet is the media. Other top targets are his political rivals like Hillary Clinton and Joe Biden. Events like impeachment inquiry and 2020 election can also be seen on his top targets.

## Insult Types
There are different types of instult type. How many uniqe insults?

In [13]:
print("Unique type of insults: ",len(df.select('insult').distinct().collect()))

Unique type of insults:  6730


However there are in total 10492 tweets. So lets take a look a at top recurring insult types.

In [14]:
insult_count=df.groupBy("insult").agg(count("*").alias("Occurence")).sort(desc("Occurence")).filter(col("Occurence")>25)

In [15]:
fig = px.bar(insult_count.toPandas(), x='insult', y='Occurence')
fig.show()

Wow his insults towards media is very high! Calling names like, "sleepy" and "crooked" pops up too.

# Tweet Date
Let's take a look at how his tweeting has evolved over time.

In [16]:
date_agg=df \
.groupBy(year("date").alias("year"),month("date").alias("month"))\
.agg(count("*").alias("Tweets"))

In [17]:
date_agg=date_agg.withColumn(
    "date",
    expr("make_date(year, month, 1)")
).sort(desc("date"))

In [18]:
fig = px.line(date_agg.toPandas(), x='date', y="Tweets")
fig.show()

He has been consistent in insulting people over the years. There are some sudden peaks that can be related with 

## Sentiment Analysis

These tweets are all insults. However could there be tweets that are positive? Since the tweets do not have any specific labels, a pretrained model for sentiment analysis will be deployed for sentiment analysis. 

In [19]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [20]:
pipeline = PretrainedPipeline("analyze_sentimentdl_use_twitter", lang="en")

analyze_sentimentdl_use_twitter download started this may take some time.
Approx size to download 935.1 MB
[OK!]


In [21]:
df_result = \
pipeline.transform(df.withColumnRenamed("tweet", "text"))

In [22]:
df_result.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      date|              target|              insult|                text|            document| sentence_embeddings|           sentiment|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|2014-10-09|      thomas-frieden|                fool|"Can you believe ...|[[document, 0, 14...|[[sentence_embedd...|[[category, 0, 14...|
|2014-10-09|      thomas-frieden|                DOPE|"Can you believe ...|[[document, 0, 14...|[[sentence_embedd...|[[category, 0, 14...|
|2015-06-16|         politicians|all talk and no a...|Big time in U.S. ...|[[document, 0, 12...|[[sentence_embedd...|[[category, 0, 12...|
|2015-06-24|          ben-cardin|It's politicians ...|Politician @Senat...|[[document, 0, 13...|[[sentence_embedd...|[[category, 0, 13...|
|2015-06-24|          neil-

In [23]:
df_result.printSchema()

root
 |-- date: date (nullable = true)
 |-- target: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- me

In [24]:
sentiment_df=df_result.select("*",df_result.sentiment.result.alias('output'))

In [25]:
sentiment_count=sentiment_df.groupBy("output").agg(count("*").alias("Number of Tweet")).sort(desc("Number of Tweet"))

In [26]:
sentiment_count.show()

+----------+---------------+
|    output|Number of Tweet|
+----------+---------------+
|[negative]|           8472|
|[positive]|           1476|
| [neutral]|            412|
|        []|            132|
+----------+---------------+



Interesting! The sentiments should all be negative as this dataset contains insults only. Let's take some of the Tweets that were predicted as positive.

In [27]:
explore_sentiment=sentiment_df.selectExpr('output[0] as output',"text")
explore_sentiment.filter(explore_sentiment.output == "positive").dropDuplicates().show(truncate=False)

+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|output  |text                                                                                                                                                                                                                                                                                    |
+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|positive|Lyin’ Ted Cruz steals foreign policy from me, and lines from Michael Douglasâ€” just another dishonest politician.

## Finding Entities 

Using a similar pretrained pipeline, entities can be detected too. This section will cover that method.

In [28]:
pipeline = PretrainedPipeline('onto_recognize_entities_bert_tiny')

onto_recognize_entities_bert_tiny download started this may take some time.
Approx size to download 30.2 MB
[OK!]


In [29]:
df_entity = \
pipeline.transform(df.withColumnRenamed("tweet", "text"))

In [30]:

df_entity.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      date|              target|              insult|                text|            document|            sentence|               token|          embeddings|                 ner|            entities|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|2014-10-09|      thomas-frieden|                fool|"Can you believe ...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 3, "C...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 32, 45, ...|
|2014-10-09|      thomas-frieden|                DOPE|"Can you believe ...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 3, "C...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 32,

In [31]:
df_entity.printSchema()

root
 |-- date: date (nullable = true)
 |-- target: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map

In [35]:
NER_df=df_entity.selectExpr('entities.result as entities',"text","NER.result as tokens").dropDuplicates()

In [37]:
NER_df.show(10,truncate=False)

+------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|entities                                  |text                                                                                                                                                                                                                                                                                 |tokens                                                                                                                                                                          |


In [52]:
NER_count=NER_df.select("entities").withColumn("entities" ,explode('entities')).groupBy("entities").agg(count("*").alias("Occurence")).sort(desc("Occurence")).filter(col("Occurence")>20)

In [53]:
fig = px.bar(NER_count.toPandas(), x='entities', y='Occurence')
fig.show()

Unlike the targets columns the most occuring entity is democrats. This might be due to the fact that he mentioned democrats when tweeting against Hilary clinton and Joe Biden. Intestingly 2016 and 2020 came up as years. This makes sense because these two years are the ones when Trump was involved in the presidential elections.

In [57]:
NER_type_count=NER_df.select("tokens").withColumn("tokens" ,explode('tokens')).groupBy("tokens").agg(count("*").alias("Occurence")).sort(desc("Occurence")).filter(col("Occurence")<5000)

In [59]:
fig = px.bar(NER_type_count.toPandas(), x='tokens', y='Occurence')
fig.show()

The plot above shows his insults mentined more persons than organizations. B-Person represents person entity. B-GPE is geopolitical entity. This may be due to the fact that he mentions U.S. a lot in his tweets.